# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
weather_data =pd.read_csv( "weather_data.csv")

del weather_data['Unnamed: 0']
weather_data.head()

,City,Latitude,Longitude,Temperature,Humidity,cloudiness,windspeed,Country,Dates
0,Bredasdorp,-34.5322,20.0403,41.00,93,0,2.30,ZA,1619829914
1,Cape Town,-33.9258,18.4232,56.73,82,0,9.22,ZA,1619829914
2,Dikson,73.5069,80.5464,16.39,96,99,7.61,RU,1619829915
3,Zhezkazgan,47.8043,67.7144,47.17,44,0,5.97,KZ,1619829915
4,Pandan,14.0453,124.1694,82.89,63,8,1.59,PH,1619829915


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
gmaps.configure(api_key=g_key)

In [57]:
locations = weather_data[['Latitude','Longitude']]
weight = weather_data['Humidity'].astype(float)

In [58]:
gmaps.configure(api_key=g_key)

In [59]:
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=weight,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=2)
#add heat layer
fig.add_layer(heat_layer)

#display map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
#deal_weather_df = pd.DataFrame(weather_data, columns = "City", "Ideal Temp", "Ideal Wind Speed", "Ideal Cloudiness")

ideal_temp = (weather_data["Temperature"]<= 80)& (weather_data["Temperature"] > 70)
ideal_windspeed = weather_data["windspeed"] < 7
ideal_cloudiness = weather_data["cloudiness"] == 0

ideal_weather_df = weather_data[ideal_temp& ideal_windspeed & ideal_cloudiness]
ideal_weather_df

,City,Latitude,Longitude,Temperature,Humidity,cloudiness,windspeed,Country,Dates
16,Atuona,-9.8000,-139.0333,79.41,68,0,3.47,PF,1619829920
42,Porto Velho,-8.7619,-63.9039,77.00,83,0,1.77,BR,1619829817
111,Saint-Pierre,-21.3393,55.4781,70.20,78,0,6.91,RE,1619829951
157,Gaya,24.7833,85.0000,73.40,60,0,6.91,IN,1619829969
159,Abu Dhabi,24.4667,54.3667,79.83,69,0,3.44,AE,1619829683
176,Manacapuru,-3.2997,-60.6206,77.00,94,0,1.16,BR,1619829975
303,Ierapetra,35.0100,25.7353,73.08,52,0,4.61,GR,1619830021
346,Parkes,-33.1333,148.1833,75.00,45,0,3.00,AU,1619830037
468,Angoche,-16.2325,39.9086,76.17,86,0,2.33,MZ,1619830084
512,Anastácio,-20.4836,-55.8069,70.90,38,0,3.91,BR,1619830105


In [46]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


hotel_list = []

params = {
    "radius": 50000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotel
for index, row in ideal_weather_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    
    try:
        hotel_list.append(hotel_name['results'][0]['name'])
    except:
        hotel_list.append(np.append)

In [47]:
hotel_list

['Atuona',
 'Porto Velho',
 'Saint-Denis',
 'Gaya',
 'Abu Dhabi',
 'Manacapuru',
 'Sitia',
 'Parkes',
 'Angoche Island',
 'Aquidauana']

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df = pd.DataFrame(ideal_weather_df)
hotel_df["Hotel Name"] = hotel_list
hotel_df

,City,Latitude,Longitude,Temperature,Humidity,cloudiness,windspeed,Country,Dates,Hotel Name
16,Atuona,-9.8000,-139.0333,79.41,68,0,3.47,PF,1619829920,Atuona
42,Porto Velho,-8.7619,-63.9039,77.00,83,0,1.77,BR,1619829817,Porto Velho
111,Saint-Pierre,-21.3393,55.4781,70.20,78,0,6.91,RE,1619829951,Saint-Denis
157,Gaya,24.7833,85.0000,73.40,60,0,6.91,IN,1619829969,Gaya
159,Abu Dhabi,24.4667,54.3667,79.83,69,0,3.44,AE,1619829683,Abu Dhabi
176,Manacapuru,-3.2997,-60.6206,77.00,94,0,1.16,BR,1619829975,Manacapuru
303,Ierapetra,35.0100,25.7353,73.08,52,0,4.61,GR,1619830021,Sitia
346,Parkes,-33.1333,148.1833,75.00,45,0,3.00,AU,1619830037,Parkes
468,Angoche,-16.2325,39.9086,76.17,86,0,2.33,MZ,1619830084,Angoche Island
512,Anastácio,-20.4836,-55.8069,70.90,38,0,3.91,BR,1619830105,Aquidauana


In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [54]:
# Add marker layer 
figure = gmaps.figure()

marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(marker)
figure.add_layer(heat_layer)
# Display figure
figure

Figure(layout=FigureLayout(height='420px'))